# Cochrane: descriptive statistics

This notebook gives a high-level overview of the descriptive statistics created for the Cochcrane project. For more in-depth exploratory data analysis of the individual datasets, see the other notebooks in this folder.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, os
from collections import Counter, defaultdict, OrderedDict
import numpy as np
from statistics import mean
from xml.etree import ElementTree
import codecs
import glob
from lxml import etree

## Published medical science

Cochrane produces systematic reviews, that attempt to give a state-of-the-art answer to a medical research question. To do this, it synthesizes (a.o.) research that is published in papers. Only a very small proportion of studies that have been considered by Cochrane (i.e. exist in its databases) have been included in reviews, meaning that the signal-to-noise ratio is very low.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-fymr{font-weight:bold;border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-0pky"></th>
    <th class="tg-fymr" colspan="2">in Review Group<br></th>
  </tr>
  <tr>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky">Yes</td>
    <td class="tg-0pky">No</td>
  </tr>
  <tr>
    <td class="tg-fymr" rowspan="2">in Review</td>
    <td class="tg-0pky">Yes</td>
    <td class="tg-0pky">158.024</td>
    <td class="tg-0pky">67.721</td>
  </tr>
  <tr>
    <td class="tg-0pky">No</td>
    <td class="tg-0pky">340.871</td>
    <td class="tg-0pky">861.494</td>
  </tr>
</table>

## Systematic reviews produced by Cochrane

Cochrane is organized in 53 Review Groups. Each of these look after a domain area for which they produce systematic reviews.

In [ ]:
# for overview file with published reviews

filename = '/data/raw/20190618_published_reviews.csv'

reviews = pd.read_csv(filename)

reviews.drop([col for col in reviews.columns if 'Unnamed' in col], axis=1, inplace=True)

In [ ]:
# General stats
# number of reviews, number of reviews per review group
print("The total numer of reviews is {}.".format(reviews["Group"].value_counts().sum()))
print("The average number of reviews per group is {}, with a \
standard deviation of {}.".format(round(reviews["Group"].value_counts().mean(),1),
                                round(reviews["Group"].value_counts().std(),1)))

The cells below show the 10 Review Groups with the highest vs. the lowest number of reviews.

In [ ]:
reviews["Group"].value_counts()[:10][::-1].plot(kind="barh", title="Number of reviews per group - top 10")

In [ ]:
reviews["Group"].value_counts()[::-1][:10].plot(kind="barh", title="Number of reviews per group - bottom 10")

Reviews can have one of six statuses. The total number of studies per status is displayed below.

In [ ]:
reviews["Publication Flag"].value_counts().iloc[::-1].plot(kind="barh", title="Status of reviews count")

The plot belows shows the distribution (in %) over Publication Flags by Review Group. It seems that most groups have a similar proportion of studies in each group, although there are some outliers. 

In [ ]:
status_by_group = reviews.groupby(["Group", "Publication Flag"]).agg({'CD Number':'count'})
status_by_group_pct = status_by_group.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()) ).reset_index()
status_by_group_pct = status_by_group_pct.pivot(index='Group', columns='Publication Flag').fillna(value=0).round(2)
plt.figure(figsize=(18,24))
plt.title("Distribution (in %) over Publication Flag by Review Group")
sns.heatmap(status_by_group_pct, annot=True)

Linking published research to reviews, the table below provides details on the number of studies included per review.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-0lax">Mean</th>
    <th class="tg-0lax">Median</th>
  </tr>
  <tr>
    <td class="tg-0lax">Included studies</td>
    <td class="tg-0lax">14.30</td>
    <td class="tg-0lax">7</td>
  </tr>
  <tr>
    <td class="tg-0lax">Excluded studies</td>
    <td class="tg-0lax">28.59</td>
    <td class="tg-0lax">14</td>
  </tr>
</table>

## PICO Annotations

Cochrane has started adding PICO annotations to reviews and other documents (papers) in its database. Automation of PICO labels could be considered a productive avenue of exploration, as having the PICO labels of a document means that it is very easy to determine whether this paper is relevant to a particular review. However, as the data is currently not sufficient to build a system for automatic PICO labelling, we will not pursue this route in the present project.

In [ ]:
os.chdir(r'/data/raw/annotations')

In [ ]:
# saving annotations to list of dicts
all_annotations = []

# keeping track of the files that fail
failed_files = []

for filename in os.listdir('.'):
    if filename.endswith('.json'):

        with open(filename, encoding='utf-8', errors='ignore') as f:
            
            try:
                # file is read: append its annotations
                file = json.load(f)
                
                for i in range(len(file['results'])):
                    annotation = file['results'][i]
                    all_annotations.append(annotation)
                
            except:
                # file cannot be read
                failed_files.append(filename)            

The bar chart below shows the top-10 of Review Groups with the largest number of reviews and studies with a PICO annotation. It seems that some Review Groups have been very active, but the number of annotations drops very quickly for the other groups.

In [ ]:
# count how many annotations exist per review group
labels_per_group = Counter()

for annotation in all_annotations:
    
    try:
        labels_per_group[annotation['reviewGroupLabel']] += 1
    
    # some annotations have the group label duplicated in the list
    except:
        
        try:
            labels_per_group[annotation['reviewGroupLabel'][0]] += 1
        
        # a small number of annotations lack reviewGroupLabel altogether
        except:
            pass
        
         
    
labels_per_group_df = pd.DataFrame.from_dict(labels_per_group, orient='index').reset_index()
labels_per_group_df.rename(columns={'index': 'review_group', 0:'annotations'}, inplace=True)

group, count = zip(*labels_per_group.most_common(10))
sns.barplot(y=list(group), x=list(count),orient="h")
plt.title("Annotations per Review Group - top 10")

The PICO chart below shows how often the PICO labels in our dataset have been used. More than half of the PICO labels have only been applied 5 times or less (meaning that there is very little training data to learn where these labels can be applied).

In [ ]:
# count the PICO labels for all variables

pico_counter = Counter()
variables = ["implicitMaterial", "sex", "outcomeClassfication", "implicitCondition", 
            "condition", "material", "interventionClassification", "age"]
no_var = 0
total_count = 0
all_labels = []

for annotation in all_annotations:
    
    for var in variables:
    
        try:
            if isinstance(annotation[var], list):
                for link in annotation[var]:
                    pico_counter[link] += 1
                    total_count += 1
                    all_labels.append(link)
            else:
                pico_counter[annotation[var]] += 1
                total_count += 1
                all_labels.append(annotation[var])
        except:
            no_var += 1
            
pico_counter_tresholds = {'x <= 5': len([pico_counter[x] for x in pico_counter if pico_counter[x] < 5]),
                         '5 < x <= 10': len([pico_counter[x] for x in pico_counter if pico_counter[x] <= 10 and pico_counter[x] > 5]),
                         '10 < x <= 100': len([pico_counter[x] for x in pico_counter if pico_counter[x] <= 100 and pico_counter[x] > 10]),
                         '100 < x <= 500': len([pico_counter[x] for x in pico_counter if pico_counter[x] <= 500 and pico_counter[x] > 100]),
                         '500 < x': len([pico_counter[x] for x in pico_counter if pico_counter[x] > 500])}

fig, ax = plt.subplots()
patches, b, c = ax.pie(pico_counter_tresholds.values(), autopct='%1.1f%%')#, colors=sns.color_palette("Blues"))
plt.legend(patches, pico_counter_tresholds.keys(), loc='best')
plt.axis('equal')
plt.title("Frequency of use of PICO labels")
plt.tight_layout()

## How many studies are in reviews?

In [ ]:
folder = '/data/raw/reviews/'
all_files = glob.glob(folder + "*.rm5")

num_included_studies = []
num_excluded_studies = []
num_awaiting_studies = []
num_ongoing_studies = []
for file_ in all_files: #over 7k files
    root = etree.parse(file_)
    included_studies = root.xpath('/COCHRANE_REVIEW/CHARACTERISTICS_OF_STUDIES/CHARACTERISTICS_OF_INCLUDED_STUDIES/INCLUDED_CHAR')
    excluded_studies = root.xpath('/COCHRANE_REVIEW/CHARACTERISTICS_OF_STUDIES/CHARACTERISTICS_OF_EXCLUDED_STUDIES/EXCLUDED_CHAR')
    awaiting_studies = root.xpath('/COCHRANE_REVIEW/CHARACTERISTICS_OF_STUDIES/CHARACTERISTICS_OF_AWAITING_STUDIES/AWAITING_CHAR')
    ongoing_studies = root.xpath('/COCHRANE_REVIEW/CHARACTERISTICS_OF_STUDIES/CHARACTERISTICS_OF_ONGOING_STUDIES/ONGOING_CHAR')
    num_included_studies.append(len(included_studies))
    num_excluded_studies.append(len(excluded_studies))
    num_awaiting_studies.append(len(awaiting_studies))
    num_ongoing_studies.append(len(ongoing_studies))

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.subplot(221)
plt.hist(num_included_studies)
plt.title("Number of included studies in a review", fontsize=20)
plt.subplot(222)
plt.hist(num_excluded_studies)
plt.title("Number of excluded studies in a review", fontsize=20)
plt.subplot(223)
plt.hist(num_awaiting_studies)
plt.title("Number of awaiting studies in a review", fontsize=20)
plt.subplot(224)
plt.hist(num_ongoing_studies)
plt.title("Number of ongoing studies in a review", fontsize=20);